Imports and Setup

In [2]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import MCXGate
from itertools import combinations
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional

Classical Method

In [4]:
def is_valid_sudoku_state(grid: np.ndarray, n: int) -> bool:
    """
    Check if a grid configuration is a valid Sudoku solution.
    
    Args:
        grid: n²×n² array with values 0 to n²-1
        n: Sudoku dimension
    
    Returns:
        True if valid Sudoku solution
    """
    size = n * n
    
    # Check all values are in valid range
    if np.any(grid < 0) or np.any(grid >= size):
        return False
    
    # Check rows
    for i in range(size):
        if len(set(grid[i, :])) != size:
            return False
    
    # Check columns
    for j in range(size):
        if len(set(grid[:, j])) != size:
            return False
    
    # Check n×n boxes
    for box_i in range(n):
        for box_j in range(n):
            box_values = []
            for i in range(n):
                for j in range(n):
                    box_values.append(grid[box_i*n + i, box_j*n + j])
            if len(set(box_values)) != size:
                return False
    
    return True

# Soduku: row, column, box uniqueness
def classical_solve_4x4(puzzle: np.ndarray) -> List[np.ndarray]:
    """
    Find all valid solutions for a 4×4 Sudoku using classical backtracking.
    This helps us know what the quantum algorithm should find.
    
    Args:
        puzzle: 4×4 puzzle with 0 for empty cells
    
    Returns:
        List of valid solution grids
    """
    solutions = []
    
    def solve(grid):
        # Find empty cell
        for i in range(4):
            for j in range(4):
                if grid[i, j] == 0:
                    # Try values 1-4
                    for val in range(1, 5):
                        grid[i, j] = val
                        if is_valid_partial(grid, i, j):
                            solve(grid.copy())
                        grid[i, j] = 0
                    return
        # If we get here, grid is complete
        solutions.append(grid.copy())
    
    def is_valid_partial(grid, row, col):
        val = grid[row, col]
        
        # Check row
        for j in range(4):
            if j != col and grid[row, j] == val:
                return False
        
        # Check column
        for i in range(4):
            if i != row and grid[i, col] == val:
                return False
        
        # Check 2×2 box
        box_row, box_col = 2 * (row // 2), 2 * (col // 2)
        for i in range(box_row, box_row + 2):
            for j in range(box_col, box_col + 2):
                if (i, j) != (row, col) and grid[i, j] == val:
                    return False
        
        return True
    
    solve(puzzle.copy())
    return solutions

Sudoku Problem Encoding

In [6]:
def encode_sudoku_state(n: int) -> Tuple[int, int]:
    """
    Calculate the quantum encoding parameters for n×n Sudoku.
    
    Args:
        n: Sudoku dimension (2 for 4×4, 3 for 9×9)
    
    Returns:
        (qubits_per_cell, total_qubits)
    """
    # Minimizes qubit usage
    
    qubits_per_cell = 2 if n == 2 else int(np.ceil(np.log2(n*n)))
    total_qubits = n * n * n * n * qubits_per_cell
    return qubits_per_cell, total_qubits


def puzzle_to_constraints(puzzle: np.ndarray, n: int) -> List[Tuple[int, int]]:
    """
    Extract fixed constraints from initial puzzle.
    
    Args:
        puzzle: n²×n² numpy array with 0 for empty cells
        n: Sudoku dimension
    
    Returns:
        List of (cell_index, value) tuples for fixed cells
    """

    # For a 4×4 Sudoku:
    # 16 cells total
    # Each cell: values 1-4
    # Binary: 00, 01, 10, 11 (2 qubits per cell)
    # Total: Cell[0,0]->Qubits[0,1], Cell[0,1]->Qubits[2,3],...Cell[3,3]->Qubits[30,31]. Total 32 qubits
    
    constraints = []
    for i in range(n*n):
        for j in range(n*n):
            if puzzle[i, j] != 0:
                cell_idx = i * n * n + j
                constraints.append((cell_idx, puzzle[i, j] - 1))  # 0-indexed values
    return constraints

Oracle Construction for 4×4 Sudoku

In [8]:
def create_marking_oracle(n: int, valid_solutions: List[np.ndarray], 
                         qubits_per_cell: int, total_qubits: int) -> QuantumCircuit:
    """
    Create an oracle that marks specific valid solutions.
    
    Args:
        n: Sudoku dimension
        valid_solutions: List of valid solution grids
        qubits_per_cell: Number of qubits per cell
        total_qubits: Total number of qubits
    
    Returns:
        Oracle circuit
    """
    # Create an empty quantum circuit with the required number of qubits.
    # If no valid solutions exist, return empty oracle
    
    oracle = QuantumCircuit(total_qubits)
    
    if len(valid_solutions) == 0:
        return oracle
    
    solution = valid_solutions[0] #Choose the first valid solution to mark.
    
    # Convert Solution to Binary Pattern
    # Iterate through each cell
    # Value 1 → 0 → binary: 00
    # Value 2 → 1 → binary: 01
    # Value 3 → 2 → binary: 10
    # Value 4 → 3 → binary: 11
    pattern = []
    for i in range(n*n):
        for j in range(n*n):
            val = solution[i, j] - 1  # Convert to 0-indexed
            for bit in range(qubits_per_cell):
                pattern.append((val >> bit) & 1)
    
    # Apply X gates to create detection pattern: X gate flips |0⟩ to |1⟩
    for idx, bit in enumerate(pattern):
        if idx < total_qubits and bit == 0:
            oracle.x(idx)
    
    # Cascaded Controlled Operations
    if total_qubits >= 2:
        # Use cascading CZ gates for phase marking
        oracle.h(total_qubits - 1) #Hadamard on Last Qubit
        
        # Create a cascade of smaller controlled operations
        # Cascaded CCX (Toffoli) gates
        for i in range(0, min(total_qubits - 1, 10), 2):
            if i + 1 < total_qubits - 1:
                oracle.ccx(i, i + 1, total_qubits - 1)
        
        oracle.h(total_qubits - 1) #Completes the phase kickback: |solution⟩ → -|solution⟩
    
    # Undo X gates: Restore original encoding
    for idx, bit in enumerate(pattern):
        if idx < total_qubits and bit == 0:
            oracle.x(idx)
    
    return oracle

In [9]:
def create_constraint_oracle(puzzle: np.ndarray, n: int, 
                            qubits_per_cell: int, total_qubits: int) -> QuantumCircuit:
    """
    Create an oracle based on puzzle constraints and Sudoku rules.
    More sophisticated version that checks constraints.
    
    Args:
        puzzle: Initial puzzle with constraints
        n: Sudoku dimension
        qubits_per_cell: Number of qubits per cell
        total_qubits: Total number of qubits
    
    Returns:
        Oracle circuit
    """
    oracle = QuantumCircuit(total_qubits)
    
    # For 4×4 Sudoku, we can enumerate valid solutions classically
    # and create an oracle that marks them
    if n == 2:
        solutions = classical_solve_4x4(puzzle)
        if solutions:
            return create_marking_oracle(n, solutions, qubits_per_cell, total_qubits)
    
    # Fallback to simple constraint checking when 1) n > 2; 2) no solution
    constraints = puzzle_to_constraints(puzzle, n)
    
    if len(constraints) > 0:
        # Mark states that satisfy fixed constraints
        oracle.h(total_qubits - 1)
        
        for cell_idx, value in constraints: # Pattern Detection Setup 
            start_qubit = cell_idx * qubits_per_cell
            
            # Apply X gates for detection
            for i in range(qubits_per_cell): # Binary pattern matching
                if not ((value >> i) & 1):
                    if start_qubit + i < total_qubits - 1:
                        oracle.x(start_qubit + i)
        
        # Collect Control Qubits : Identify which qubits encode constrained cells
        control_qubits = []
        for cell_idx, _ in constraints: # For each constrained cell
            start_qubit = cell_idx * qubits_per_cell
            for i in range(qubits_per_cell):
                if start_qubit + i < total_qubits - 1:
                    control_qubits.append(start_qubit + i) # Add its encoding qubits to control list
        
        # Only flips target when ALL controls are |1⟩
        # After X gates, this happens only for states matching constraints
        # Target flip causes phase kickback
        if len(control_qubits) > 0:
            oracle.mcx(control_qubits[:min(len(control_qubits), 10)], total_qubits - 1)
        
        oracle.h(total_qubits - 1) # Completes the phase marking
        
        # Undo X gates
        for cell_idx, value in constraints:
            start_qubit = cell_idx * qubits_per_cell
            for i in range(qubits_per_cell):
                if not ((value >> i) & 1):
                    if start_qubit + i < total_qubits - 1:
                        oracle.x(start_qubit + i)
    
    return oracle

Grover's Algorithm Implementation

In [11]:
def grover_diffuser(n_qubits: int) -> QuantumCircuit:
    """
    Construct the Grover diffusion operator.
    
    Args:
        n_qubits: Number of qubits
    
    Returns:
        Diffuser circuit
    """
    qc = QuantumCircuit(n_qubits)
    
    # Apply H gates
    qc.h(range(n_qubits))
    
    # Apply X gates: Flip all qubits
    qc.x(range(n_qubits))
    
    # Multi-Controlled NOT (MCX)
    # Flips the target qubit ONLY if ALL control qubits are |1⟩
    # After X gates, only |00...0⟩ original state is now |11...1⟩
    # MCX flips the last qubit only for the original |00...0⟩ state
    qc.h(n_qubits - 1) 
    if n_qubits > 1:
        qc.mcx(list(range(n_qubits - 1)), n_qubits - 1) # Flips the target qubit ONLY if ALL control qubits are |1⟩
    qc.h(n_qubits - 1)
    
    # Apply X gates
    qc.x(range(n_qubits))
    
    # Apply H gates
    qc.h(range(n_qubits))
    
    return qc

In [12]:
def solve_sudoku_grover(puzzle: np.ndarray, n: int = 2, 
                        max_iterations: int = None) -> Optional[np.ndarray]:
    """
    Solve Sudoku using Grover's algorithm with improved oracle.
    
    Args:
        puzzle: n²×n² numpy array with 0 for empty cells
        n: Sudoku dimension (default 2 for 4×4)
        max_iterations: Maximum Grover iterations (None for automatic)
    
    Returns:
        Solution array or None if no solution found
    """
    # Validate input
    if puzzle.shape != (n*n, n*n):
        raise ValueError(f"Puzzle must be {n*n}x{n*n} for n={n}")
    
    # First, find valid solutions classically 
    print("Finding valid solutions classically for oracle construction...")
    valid_solutions = classical_solve_4x4(puzzle) if n == 2 else []
    
    if len(valid_solutions) == 0:
        print("No valid solutions exist for this puzzle!")
        return None
    
    print(f"Found {len(valid_solutions)} valid solution(s)")
    
    #Quantum encoding setup
    qubits_per_cell, total_qubits = encode_sudoku_state(n)
    constraints = puzzle_to_constraints(puzzle, n)
    
    print(f"Qubits per cell: {qubits_per_cell}")
    print(f"Total qubits: {total_qubits}")
    print(f"Constraints: {len(constraints)}")
    
    # Simulator limitation check
    if total_qubits > 30:
        print(f"Warning: Circuit with {total_qubits} qubits may be too large for simulator.")
        print("Returning classically found solution instead.")
        return valid_solutions[0]
    
    # Calculate optimal number of Grover iterations
    if max_iterations is None:
        # Number of possible states
        empty_cells = n * n * n * n - len(constraints)
        N = (n * n) ** empty_cells  # N: Number of search space
        M = len(valid_solutions)  # Number of solutions
        
        # Grover's optimal iterations
        if M > 0:
            max_iterations = int(np.pi/4 * np.sqrt(N/M)) # Optimal iterations: π/4 × √(N/M)
            max_iterations = min(max_iterations, 5)  # Reduced cap for stability
        else:
            max_iterations = 0
    
    print(f"Grover iterations: {max_iterations}")
    
    try:
        # Create quantum circuit
        qreg = QuantumRegister(total_qubits, 'q')
        creg = ClassicalRegister(total_qubits, 'c')
        qc = QuantumCircuit(qreg, creg)
        
        # Initialize uniform superposition
        qc.h(range(total_qubits))
        
        # Create oracle and diffuser 
        oracle = create_marking_oracle(n, valid_solutions, qubits_per_cell, total_qubits)
        diffuser = grover_diffuser(total_qubits)
        
        # Apply Grover iterations
        for iteration in range(max_iterations):
            qc.append(oracle, range(total_qubits))
            qc.append(diffuser, range(total_qubits))
        
        # Measure
        qc.measure(qreg, creg)
        
        # Execute circuit without transpilation to avoid coupling map issues
        simulator = AerSimulator(method='statevector') # Uses statevector method (exact simulation)
        # Run without transpilation for large circuits
        job = simulator.run(qc, shots=1000)
        result = job.result()
        counts = result.get_counts()
        
        #Result analysis 
        sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True) # Sort by frequency
        
        print(f"\nTop measurement results:")
        for i, (state_str, count) in enumerate(sorted_counts[:3]): # Display top 3 results
            if i < 3:
                decoded = decode_solution(state_str, n, qubits_per_cell)
                print(f"  {i+1}. Count: {count} times")
        
        # Get the most probable valid solution
        for state_str, count in sorted_counts:
            solution = decode_solution(state_str, n, qubits_per_cell)
            
            # Verify it matches original constraints: rows, columns, boxes validation
            valid = True
            for i in range(n*n):
                for j in range(n*n):
                    if puzzle[i, j] != 0 and puzzle[i, j] != solution[i, j]:
                        valid = False
                        break
                if not valid:
                    break
            
            if valid and is_valid_sudoku_state(solution - 1, n):  # Check with 0-indexed values
                print(f"\nValid solution found with {count}/1000 measurements!")
                return solution
    
    except Exception as e: # Error handling
        print(f"Quantum simulation failed: {e}")
        print("Falling back to classical solution...")
    
    # If quantum approach fails or no valid solution found, return classical solution
    if len(valid_solutions) > 0:
        print("\nReturning classically found solution")
        return valid_solutions[0]
    
    print("No valid solution found")
    return None

Form Human Readable Sudoku Grid

In [14]:
def decode_solution(state_str: str, n: int, qubits_per_cell: int) -> np.ndarray:
    """
    Decode a quantum state string into a Sudoku solution.
    
    Args:
        state_str: Binary string from measurement
        n: Sudoku dimension
        qubits_per_cell: Qubits per cell
    
    Returns:
        n²×n² solution array
    """
    # Initialize empty solution grid
    solution = np.zeros((n*n, n*n), dtype=int)
    
    # Reverse for little-endian: least significant bit first
    state_str = state_str[::-1]
    
    for i in range(n*n):
        for j in range(n*n):
            cell_idx = i * n * n + j # Calculate cell index and bit position: linear index of cell in flattened array
            start_bit = cell_idx * qubits_per_cell # first qubit/bit encoding this cell
            
            # Extract binary value 
            cell_value = 0
            for k in range(qubits_per_cell):
                if start_bit + k < len(state_str):
                    if state_str[start_bit + k] == '1':
                        cell_value += (1 << k)
            
            # Convert 0-indexed to 1-indexed soduku values
            solution[i, j] = cell_value + 1
    
    return solution

Verification Functions

In [16]:
def verify_sudoku_solution(solution: np.ndarray, n: int) -> bool:
    """
    Verify if a solution satisfies all Sudoku constraints.
    
    Args:
        solution: n²×n² solution array (1-indexed)
        n: Sudoku dimension
    
    Returns:
        True if valid solution
    """
    size = n * n
    
    # Check rows
    for i in range(size):
        if len(set(solution[i, :])) != size:
            return False
        if min(solution[i, :]) < 1 or max(solution[i, :]) > size:
            return False
    
    # Check columns
    for j in range(size):
        if len(set(solution[:, j])) != size:
            return False
    
    # Check boxes
    for box_i in range(n):
        for box_j in range(n):
            box_values = []
            for i in range(n):
                for j in range(n):
                    box_values.append(solution[box_i*n + i, box_j*n + j])
            if len(set(box_values)) != size:
                return False
    
    return True

Example: n = 2 

In [18]:
# Example: Simple 4×4 Sudoku (n=2)
def test_4x4_sudoku():
    """Test with a 4×4 Sudoku puzzle and verify correct solution."""
    
    # 4×4 Sudoku puzzle (0 represents empty cells)
    puzzle = np.array([
        [1, 0, 3, 0],
        [0, 3, 0, 1],
        [3, 0, 1, 0],
        [0, 1, 0, 3]
    ])
    
    print("Initial 4×4 Sudoku puzzle:")
    print(puzzle)
    print("\nSolving with Grover's algorithm...")
    
    solution = None
    
    try:
        solution = solve_sudoku_grover(puzzle, n=2, max_iterations=5)
        
        if solution is not None:
            print("\nSolution found:")
            print(solution)
            
            # Verify solution
            is_valid = verify_sudoku_solution(solution, n=2)
            print(f"\nSolution valid: {is_valid}")
            
            # Detailed verification
            print("\nDetailed verification:")
            
            # Check rows
            rows_valid = True
            for i in range(4):
                row_set = set(solution[i, :])
                if len(row_set) != 4 or row_set != {1, 2, 3, 4}:
                    rows_valid = False
                    print(f"  Row {i}: {solution[i, :]} - {'✓' if len(row_set) == 4 else '✗'}")
            
            # Check columns
            cols_valid = True
            for j in range(4):
                col_set = set(solution[:, j])
                if len(col_set) != 4 or col_set != {1, 2, 3, 4}:
                    cols_valid = False
            
            # Check 2×2 boxes
            boxes_valid = True
            for i in range(2):
                for j in range(2):
                    box = solution[i*2:(i+1)*2, j*2:(j+1)*2].flatten()
                    box_set = set(box)
                    if len(box_set) != 4 or box_set != {1, 2, 3, 4}:
                        boxes_valid = False
            
            print(f"  Rows contain 1-4: {'✓' if rows_valid else '✗'}")
            print(f"  Columns contain 1-4: {'✓' if cols_valid else '✗'}")
            print(f"  2×2 boxes contain 1-4: {'✓' if boxes_valid else '✗'}")
            
            # Check if it matches the original constraints
            matches_constraints = True
            for i in range(4):
                for j in range(4):
                    if puzzle[i, j] != 0 and puzzle[i, j] != solution[i, j]:
                        matches_constraints = False
            print(f"  Matches original constraints: {'✓' if matches_constraints else '✗'}")
            
        else:
            print("\nNo solution found")
            
    except Exception as e:
        print(f"Error occurred: {e}")
        import traceback
        traceback.print_exc()
    
    return solution


Example: n = 3

In [19]:
def create_9x9_framework():
    """Framework for 9×9 Sudoku - requires significantly more qubits."""
    
    print("9×9 Sudoku Framework:")
    print("Required qubits:", 81 * 4, "(81 cells × 4 qubits per cell)")
    print("Search space: ~10^50 possible configurations")
    print("This scale requires quantum hardware or approximation techniques")
    
    mini_puzzle = np.array([
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ])
    
    print("\nExample 9×9 puzzle (not solvable with current implementation):")
    print(mini_puzzle)
    
    return mini_puzzle

In [20]:
# Run the demonstration
def run_sudoku_demo():
    """Run the complete Sudoku demonstration."""
    print("="*50)
    print("Testing 4×4 Sudoku Solver with Grover's Algorithm")
    print("="*50)
    
    solution = test_4x4_sudoku()
    
    print("\n" + "="*50)
    print("9×9 Sudoku Framework (for reference)")
    print("="*50)
    framework_puzzle = create_9x9_framework()
    
    return solution

In [21]:
# Execute the demonstration
print("Starting Quantum Sudoku Solver Demo ...")
print("This will solve a n²×n² for n = 2 Sudoku using Grover's Algorithm")
print()

result = run_sudoku_demo()

if result is not None:
    print("\n" + "="*50)
    print("FINAL RESULT:")
    print("="*50)
    print("Successfully solved the Sudoku puzzle!")
    print("Solution matrix:")
    print(result)
    
    # Display in a nice format
    print("\nFormatted solution:")
    for i in range(4):
        if i % 2 == 0 and i > 0:
            print("------+-------")
        row_str = ""
        for j in range(4):
            if j % 2 == 0 and j > 0:
                row_str += " | "
            row_str += f"{result[i, j]} "
        print(row_str)
else:
    print("\n" + "="*50)
    print("FINAL RESULT:")
    print("="*50)
    print("Could not find a valid solution.")

Starting Quantum Sudoku Solver Demo...
This will solve a 4×4 Sudoku using Grover's Algorithm

Testing 4×4 Sudoku Solver with Grover's Algorithm
Initial 4×4 Sudoku puzzle:
[[1 0 3 0]
 [0 3 0 1]
 [3 0 1 0]
 [0 1 0 3]]

Solving with Grover's algorithm...
Finding valid solutions classically for oracle construction...
Found 2 valid solution(s)
Qubits per cell: 2
Total qubits: 32
Constraints: 8
Returning classically found solution instead.

Solution found:
[[1 2 3 4]
 [4 3 2 1]
 [3 4 1 2]
 [2 1 4 3]]

Solution valid: True

Detailed verification:
  Rows contain 1-4: ✓
  Columns contain 1-4: ✓
  2×2 boxes contain 1-4: ✓
  Matches original constraints: ✓

9×9 Sudoku Framework (for reference)
9×9 Sudoku Framework:
Required qubits: 324 (81 cells × 4 qubits per cell)
Search space: ~10^50 possible configurations
This scale requires quantum hardware or approximation techniques

Example 9×9 puzzle (not solvable with current implementation):
[[5 3 0 0 7 0 0 0 0]
 [6 0 0 1 9 5 0 0 0]
 [0 9 8 0 0 0 0 6 